In [21]:
import sys
sys.path.insert(0, '../')

In [33]:
from pprint import pprint
from collections import Counter
from sklearn.metrics import accuracy_score

import rdflib
import wlkernel
import math
import numpy as np

In [23]:
rdf_graph = rdflib.Graph().parse('../data/aifbfixed_complete.n3', format='n3')

In [24]:
affiliation_uri = rdflib.URIRef('http://swrc.ontoware.org/ontology#affiliation')
instances = [ (t[0], t[2]) for t in rdf_graph.triples( (None, affiliation_uri, None) ) ]
instances = [(x, _) for (x, _) in instances if x != rdflib.term.URIRef('http://www.aifb.uni-karlsruhe.de/Personen/viewPersonOWL/id1959instance')]
assert len(instances) == len(set(instances))
print(len(instances))

177


In [25]:
instances_id = [str(x[0]) for x in instances]
instances_class = [str(x[1]) for x in instances]
# pprint(Counter(instances_class).most_common())

In [26]:
employs_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#employs')
head_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#head')
member_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#member')

#predicates_to_remove = [employs_uri, head_uri, member_uri, affiliation_uri]
predicates_to_remove = [employs_uri, affiliation_uri]

for predicate in predicates_to_remove:
    rdf_graph.remove( (None, predicate, None) )

In [27]:
triples = ((str(s), str(p), str(o)) for s, p, o in rdf_graph)
g = wlkernel.WLRDFGraph(triples=triples, instances=instances_id, max_depth=2)

In [28]:
kernel_matrix = wlkernel.wlrdf_kernel_matrix(graph=g, instances=instances_id, iterations=1)

In [29]:
from sklearn import svm
svm_clf = svm.SVC(kernel='precomputed')
svm_clf.fit(kernel_matrix, instances_class)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='precomputed', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [30]:
y_pred = svm_clf.predict(kernel_matrix)

In [31]:
accuracy_score(instances_class, y_pred)

0.6214689265536724

In [35]:
np.array(list(range(len(kernel_matrix))))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176])

In [39]:
svm_clf.predict(np.array(list(range(len(kernel_matrix)))).reshape(1, -1))

array(['http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id2instance'],
      dtype='<U86')